In [1]:
mountedDB = {}

In [2]:
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
!pip install easydict

Writing to /home/mw/.config/pip/pip.conf
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You are using pip version 18.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import time
import torch.backends.cudnn as cudnn
from torch import nn
from easydict import EasyDict as edict
from models import SRResNet
from datasets import SRDataset
from utils import *

In [4]:
# config
config = edict()
config.csv_folder = '/home/mw/project/SRDataset'
config.HR_data_folder = '/home/mw/input/dataset76853/DIV2K_train_HR/DIV2K_train_HR'
config.LR_data_folder = '/home/mw/input/dataset76853/DIV2K_train_LR_bicubic_X4/DIV2K_train_LR_bicubic/X4'
config.crop_size = 96
config.scaling_factor = 4

# Model parameters
large_kernel_size = 9  # kernel size of the first and last convolutions which transform the inputs and outputs
small_kernel_size = 3  # kernel size of all convolutions in-between, i.e. those in the residual and subpixel convolutional blocks
n_channels = 64  # number of channels in-between, i.e. the input and output channels for the residual and subpixel convolutional blocks
n_blocks = 16  # number of residual blocks

# Learning parameters
config.checkpoint = "/home/mw/project/checkpoint_srresnet.pth.tar"  # path to model checkpoint, None if none
config.batch_size = 16  # batch size
config.start_epoch = 10  # start at this epoch
config.epochs = 20
config.workers = 4
config.beta = 1e-3  # the coefficient to weight the adversarial loss in the perceptual loss
config.print_freq = 50
config.lr = 1e-4
config.grad_clip = None    # clip if gradients are exploding

# Default device
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

In [5]:
if config.checkpoint is None:
    model = SRResNet(large_kernel_size=large_kernel_size, small_kernel_size=small_kernel_size,
                        n_channels=n_channels, n_blocks=n_blocks, scaling_factor=config.scaling_factor)
    # Initialize the optimizer
    optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                    lr=config.lr)

else:
    checkpoint = torch.load(config.checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']

In [6]:
# Move to default device
model = model.to(config.device)
criterion = nn.MSELoss().to(config.device)

In [7]:
# Custom dataloaders
train_dataset = SRDataset(split='train', config=config)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.workers,
                                            pin_memory=True)  # note that we're passing the collate function here

In [8]:
def train(train_loader, model, criterion, optimizer, epoch):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: content loss function (Mean Squared-Error loss)
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    model.train()  # training mode enables batch normalization

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()

    # Batches
    for i, (lr_imgs, hr_imgs) in enumerate(train_loader):
        data_time.update(time.time() - start)

        # Move to default device
        lr_imgs = lr_imgs.to(config.device)  # (batch_size (N), 3, 24, 24), imagenet-normed
        hr_imgs = hr_imgs.to(config.device)  # (batch_size (N), 3, 96, 96), in [-1, 1]

        # Forward prop.
        sr_imgs = model(lr_imgs)  # (N, 3, 96, 96), in [-1, 1]

        # Loss
        loss = criterion(sr_imgs, hr_imgs)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        # Keep track of loss
        losses.update(loss.item(), lr_imgs.size(0))

        # Keep track of batch time
        batch_time.update(time.time() - start)

        # Reset start time
        start = time.time()

        # Print status
        if i % config.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]----'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})----'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})----'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(epoch, i, len(train_loader),
                                                                    batch_time=batch_time,
                                                                    data_time=data_time, loss=losses))
    del lr_imgs, hr_imgs, sr_imgs  # free some memory since their histories may be stored


In [9]:
# Epochs
for epoch in range(config.start_epoch, config.epochs):
    # One epoch's training
    train(train_loader=train_loader,
              model=model,
              criterion=criterion,
              optimizer=optimizer,
              epoch=epoch)
    # Save checkpoint
    torch.save({'epoch': epoch,
                'model': model,
                'optimizer': optimizer},
                'checkpoint_srresnet.pth.tar')

Epoch: [10][0/5000]----Batch Time 3.351 (3.351)----Data Time 1.806 (1.806)----Loss 0.0019 (0.0019)
Epoch: [10][50/5000]----Batch Time 0.096 (0.431)----Data Time 0.041 (0.340)----Loss 0.0018 (0.0026)
Epoch: [10][100/5000]----Batch Time 0.702 (0.428)----Data Time 0.631 (0.349)----Loss 0.0036 (0.0027)
Epoch: [10][150/5000]----Batch Time 0.531 (0.428)----Data Time 0.472 (0.354)----Loss 0.0058 (0.0026)
Epoch: [10][200/5000]----Batch Time 0.452 (0.421)----Data Time 0.395 (0.350)----Loss 0.0014 (0.0026)
Epoch: [10][250/5000]----Batch Time 0.056 (0.423)----Data Time 0.000 (0.353)----Loss 0.0039 (0.0026)
Epoch: [10][300/5000]----Batch Time 0.176 (0.419)----Data Time 0.115 (0.351)----Loss 0.0021 (0.0027)
Epoch: [10][350/5000]----Batch Time 0.065 (0.421)----Data Time 0.000 (0.353)----Loss 0.0019 (0.0027)
Epoch: [10][400/5000]----Batch Time 0.063 (0.419)----Data Time 0.000 (0.353)----Loss 0.0012 (0.0027)
Epoch: [10][450/5000]----Batch Time 0.055 (0.420)----Data Time 0.000 (0.354)----Loss 0.0012 (0